I think the NN will work best with a morgan fingerprint. I will code the morgan FP into a new dataframe for us to use for this. First, I will try to make the NN work with the featurised data.

In [21]:
import itertools
import numpy as np
import pandas as pd
import statsmodels.api as sm
from matplotlib import pyplot as plt
import seaborn as sns
import torch
import rdkit
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
# Render the figure in a notebook:
%matplotlib inline  

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import plot_tree

from sklearn import metrics

from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torch.nn as nn
import torch.nn.functional as F

from torchmetrics.classification import Accuracy
from torchmetrics.classification import Recall
from torchmetrics import R2Score
from torchmetrics import MeanAbsoluteError

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

In [22]:
# x_train = pd.read_csv('../no_nontoxic_nans_X_train.csv',index_col=0)
# y_train = pd.read_csv('../no_nontoxic_nans_y_train.csv',index_col=0)

# x_val = pd.read_csv('../no_nontoxic_nans_x_val.csv')
# y_val = pd.read_csv('../no_nontoxic_nans_y_val.csv',index_col=0)

# x_test = pd.read_csv('../no_nontoxic_nans_x_test.csv')
# y_test = pd.read_csv('../no_nontoxic_nans_y_test.csv',index_col=0)

df = pd.read_csv('../../tox21.csv')

In [23]:
df_dropped_Al = df.drop([1322, 2290,2297,3558, 4565,4649, 5538, 6723, 7830])
df_dropped_Al_reindex = df_dropped_Al.reset_index(drop=True)
df_dropped_Al_reindex

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX3021,CCOc1ccc2nc(S(N)(=O)=O)sc2c1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3020,CCN1C(=O)NC(c2ccccc2)C1=O
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,TOX3024,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3027,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX20800,CC(O)(P(=O)(O)O)P(=O)(O)O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7817,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.0,TOX25184,CCCNCC(O)COc1ccccc1C(=O)CCc1ccccc1
7818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,TOX2725,CCOc1nc2cccc(C(=O)O)c2n1Cc1ccc(-c2ccccc2-c2nnn...
7819,1.0,1.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,TOX2370,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...
7820,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX2371,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...


In [24]:
#need to make a thing with dropped Al and only the data 

df_only_data = df_dropped_Al_reindex.drop(columns=['mol_id', 'smiles']) #drop the columns.

toxicnans = [] #compounds that are toxic, and contain a nan value, containing the index
nontoxicnans = [] # compounds that aren't toxic, and contain a nan value.
nonans = [] #compounds w/ no nans, toxic or not toxic
toxics = []

for row in range(len(df_only_data)): #select each row
    hasnan = False
    istoxic = False
    for i in df_only_data.iloc[row]: #select each row
        # print(row, ':',i,type(i))
        if i != 0:
            if i != 1:
                hasnan = True
        if i == 1.0:
            istoxic = True
    if hasnan == True and istoxic == True:
        toxicnans.append(row)
    elif hasnan == True and istoxic == False:
        nontoxicnans.append(row)
    elif hasnan == False:
        nonans.append(row)
    if istoxic == True:
        toxics.append(row)

df_toxicity = pd.DataFrame(df_dropped_Al_reindex['smiles']) #make new dataframe that will be the final one containing the toxicity column)
df_toxicity['mol_id'] = pd.DataFrame(df_dropped_Al_reindex['mol_id'])
df_toxicity['toxic']=0
df_toxicity

,smiles,mol_id,toxic
0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,TOX3021,0
1,CCN1C(=O)NC(c2ccccc2)C1=O,TOX3020,0
2,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...,TOX3024,0
3,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C,TOX3027,0
4,CC(O)(P(=O)(O)O)P(=O)(O)O,TOX20800,0
...,...,...,...
7817,CCCNCC(O)COc1ccccc1C(=O)CCc1ccccc1,TOX25184,0
7818,CCOc1nc2cccc(C(=O)O)c2n1Cc1ccc(-c2ccccc2-c2nnn...,TOX2725,0
7819,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...,TOX2370,0
7820,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...,TOX2371,0


In [25]:
df_no_nontoxic_nans = df_toxicity.drop(nontoxicnans) #now drop the non toxic nans, they are unsafe because we can't be sure that nan isn't hiding toxicity

In [26]:
for i in toxicnans: #iterate through the indexes of toxicnans, and sets their toxicity to 1
    df_toxicity.loc[i, 'toxic'] = 1
df_toxicity

,smiles,mol_id,toxic
0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,TOX3021,1
1,CCN1C(=O)NC(c2ccccc2)C1=O,TOX3020,0
2,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...,TOX3024,0
3,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C,TOX3027,0
4,CC(O)(P(=O)(O)O)P(=O)(O)O,TOX20800,0
...,...,...,...
7817,CCCNCC(O)COc1ccccc1C(=O)CCc1ccccc1,TOX25184,0
7818,CCOc1nc2cccc(C(=O)O)c2n1Cc1ccc(-c2ccccc2-c2nnn...,TOX2725,0
7819,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...,TOX2370,1
7820,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...,TOX2371,0


In [27]:
for i in toxics: #iterate through the indexes of all other toxics, and sets their toxicity to 1
    df_no_nontoxic_nans.loc[i, 'toxic'] = 1
df_no_nontoxic_nans = df_no_nontoxic_nans.reset_index(drop=True) #this is our final dataframe.
df_no_nontoxic_nans

,smiles,mol_id,toxic
0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,TOX3021,1
1,CC(O)(P(=O)(O)O)P(=O)(O)O,TOX20800,0
2,O=S(=O)(Cl)c1ccccc1,TOX6619,0
3,O=C(O)Cc1cc(I)c(Oc2ccc(O)c(I)c2)c(I)c1,TOX25232,1
4,CC(C)COC(=O)C(C)C,TOX6612,0
...,...,...,...
5263,O=C(O[C@@H]1Cc2c(O)cc(O)cc2O[C@@H]1c1cc(O)c(O)...,TOX567,1
5264,c1cnc(N2CCN(Cc3ccc4c(c3)OCO4)CC2)nc1,TOX25188,0
5265,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...,TOX2370,1
5266,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...,TOX2371,1


In [28]:
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*') 

fp_list = []

for smiles in df_no_nontoxic_nans['smiles']:
    # print(smiles)
    mol = Chem.MolFromSmiles(smiles)
    fpgen = AllChem.GetMorganGenerator(radius=2)
    fp = fpgen.GetFingerprintAsNumPy(mol)
    fp_list.append(fp)
    

In [29]:
x_raw = pd.DataFrame(fp_list)
x_raw

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5263,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5264,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5265,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5266,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
x_raw['smiles'] = df_no_nontoxic_nans['smiles']

In [31]:
x_raw['toxic'] = df_no_nontoxic_nans['toxic']
x_raw

,0,1,2,3,4,5,6,7,8,9,...,2040,2041,2042,2043,2044,2045,2046,2047,smiles,toxic
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,CC(O)(P(=O)(O)O)P(=O)(O)O,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,O=S(=O)(Cl)c1ccccc1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,O=C(O)Cc1cc(I)c(Oc2ccc(O)c(I)c2)c(I)c1,1
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,CC(C)COC(=O)C(C)C,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5263,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,O=C(O[C@@H]1Cc2c(O)cc(O)cc2O[C@@H]1c1cc(O)c(O)...,1
5264,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,c1cnc(N2CCN(Cc3ccc4c(c3)OCO4)CC2)nc1,0
5265,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...,1
5266,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...,1


In [32]:
x_raw.to_csv('MorganFingerprint_unsplit.csv')

In [34]:
x, x_test, y, y_test = train_test_split(x_raw.drop(columns=['smiles', 'toxic']), x_raw[['smiles', 'toxic']],test_size=0.15,random_state=0) #split the test off first
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size = 0.15/0.85,random_state=0) #then split this again.

In [38]:
X_train.to_csv('MorganFingerprint_x_train.csv')
x_test.to_csv('MorganFingerprint_x_test.csv')
X_val.to_csv('MorganFingerprint_x_val.csv')

y_train.to_csv('MorganFingerprint_y_train.csv')
y_test.to_csv('MorganFingerprint_y_test.csv')
y_val.to_csv('MorganFingerprint_y_val.csv')
